<a href="https://colab.research.google.com/github/TheMortalSea/TheMortalSea.github.io/blob/main/W03_Exercise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install duckdb leafmap

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 515.2/515.2 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.7/213.7 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 61.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.6/108.6 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 50.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 717.6/717.6 kB 30.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.2/194.2 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.4/72.4 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 48.4 MB/s eta 0:00:00


In [2]:
import duckdb
import leafmap

In [3]:
url = "https://open.gishub.org/data/duckdb/nyc_data.db.zip"
leafmap.download_file(url, unzip=True)

Downloading...
From: https://open.gishub.org/data/duckdb/nyc_data.db.zip
To: /content/nyc_data.db.zip
100%|██████████| 8.60M/8.60M [00:00<00:00, 20.2MB/s]


Extracting files...


'/content/nyc_data.db.zip'

In [4]:
con = duckdb.connect('nyc_data.db')

In [5]:
con.install_extension('spatial')
con.load_extension('spatial')

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [6]:
con.sql("SHOW TABLES;")

┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ nyc_census_blocks   │
│ nyc_homicides       │
│ nyc_neighborhoods   │
│ nyc_streets         │
│ nyc_subway_stations │
└─────────────────────┘

In [7]:
# What is the geometry value for the street named 'Atlantic Commons'?

con.sql("""

SELECT name, ST_AsText(geom)
  FROM nyc_streets
  WHERE name = 'Atlantic Commons';

""")

┌──────────────────┬──────────────────────────────────────────────────────────────────────────────────────────────┐
│       NAME       │                                       st_astext(geom)                                        │
│     varchar      │                                           varchar                                            │
├──────────────────┼──────────────────────────────────────────────────────────────────────────────────────────────┤
│ Atlantic Commons │ MULTILINESTRING ((586781.7015777241 4504202.153143394, 586863.5196448397 4504215.988170098)) │
└──────────────────┴──────────────────────────────────────────────────────────────────────────────────────────────┘

In [8]:
# What neighborhood and borough is Atlantic Commons in?

con.sql("""
SELECT name, boroname
FROM nyc_neighborhoods
WHERE ST_Intersects(geom, ST_GeomFromText('MULTILINESTRING ((586781.7015777241 4504202.153143394, 586863.5196448397 4504215.988170098))'));
""")

┌────────────┬──────────┐
│    NAME    │ BORONAME │
│  varchar   │ varchar  │
├────────────┼──────────┤
│ Fort Green │ Brooklyn │
└────────────┴──────────┘

In [9]:
# What streets does Atlantic Commons join with?

con.sql("""
SELECT name,
FROM nyc_streets
WHERE ST_Touches(geom, ST_GeomFromText('MULTILINESTRING ((586781.7015777241 4504202.153143394, 586863.5196448397 4504215.988170098))'));
""")

┌───────────────┐
│     NAME      │
│    varchar    │
├───────────────┤
│ S Oxford St   │
│ Cumberland St │
└───────────────┘

In [15]:
# Approximately how many people live on (within 50 meters of) Atlantic Commons?
con.sql("""
SELECT COLUMN_NAME
FROM INFORMATION_SCHEMA.COLUMNS
WHERE TABLE_NAME = N'nyc_census_blocks'
""")

┌─────────────┐
│ column_name │
│   varchar   │
├─────────────┤
│ BLKID       │
│ POPN_TOTAL  │
│ POPN_WHITE  │
│ POPN_BLACK  │
│ POPN_NATIV  │
│ POPN_ASIAN  │
│ POPN_OTHER  │
│ BORONAME    │
│ geom        │
└─────────────┘

In [16]:
con.sql("""
SELECT SUM(POPN_TOTAL)
FROM nyc_census_blocks
WHERE ST_DWithin(geom, ST_GeomFromText('MULTILINESTRING ((586781.7015777241 4504202.153143394, 586863.5196448397 4504215.988170098))'), 50);
""")

┌─────────────────┐
│ sum(POPN_TOTAL) │
│     int128      │
├─────────────────┤
│            1438 │
└─────────────────┘